# Enterprise AI Control Plane: Snowflake Notebook Demo

This notebook provides a hands-on demonstration of the Enterprise AI Control Plane V1, running entirely within a Snowflake Notebook.

**Goal:** Showcase how a Control Plane can orchestrate agents, policies, and workflows to create a governed, auditable AI system.

**Scenario:** We'll simulate a model drift detection event and watch the Control Plane coordinate a response between an MLOps platform (Atlas) and a reasoning agent (Intelligence Agent).

## 1. Setup and Imports

First, let's import the necessary components from our `control_plane` library. Ensure the `src` directory is added to your Snowflake Notebook's classpath.

In [ ]:
from control_plane.config import ControlPlaneConfig
from control_plane.registry import ControlPlaneRegistry
from control_plane.agents import AtlasAgent, IntelligenceAgent
from control_plane.policies import Policy, PolicyEngine
from control_plane.workflows import WorkflowEngine
from control_plane.observability import RunLogger

# Get the active Snowpark session from the notebook environment
session = get_active_session()

## 2. Configuration & Observability

We define a simple configuration and set up a `RunLogger`. In a real system, this logger would write to a persistent table in Snowflake, giving us a complete audit trail of every Control Plane action.

In [ ]:
config = ControlPlaneConfig(
    environment="dev",
    run_log_table="CONTROL_PLANE_DB.CONTROL_PLANE_MONITORING.RUN_LOG",
)

logger = RunLogger(session=session, run_log_table=config.run_log_table)

## 3. Registering Agents

The Control Plane needs to know what agents it can command. Here, we register two agents:

- **AtlasAgent**: Represents our MLOps platform.
- **IntelligenceAgent**: Represents a reasoning engine.

For this demo, we use a `DummyIntelligenceAgentImpl` to simulate the reasoning agent's behavior without requiring a real model.

In [ ]:
# Stub implementation for the Intelligence Agent
class DummyIntelligenceAgentImpl:
    def run(self, query: str):
        return {
            "summary": f"This is a dummy explanation for the query: {query}",
            "trace": []
        }

intel_impl = DummyIntelligenceAgentImpl()

intel_agent = IntelligenceAgent(
    name="snowflake_intel_agent_v2",
    description="Multi-modal reasoning agent on Snowflake.",
    session=session,
    agent_impl=intel_impl,
)

atlas_agent = AtlasAgent(
    name="atlas_mlops",
    description="Atlas MLOps control functions.",
    session=session,
)

## 4. Defining Policies

Policies are the rules of the road for your AI system. They define the conditions under which the Control Plane should act.

Here, we create a simple policy for model drift: if `observed_drift` exceeds `0.15`, trigger a retraining and notify the owner.

In [ ]:
drift_policy = Policy(
    name="DRIFT_POLICY_FRAUD_MODEL",
    description="Trigger retrain when observed drift exceeds max_drift.",
    conditions={"max_drift": 0.15},
    actions=["trigger_retrain", "notify_owner"],
)

policy_engine = PolicyEngine(
    policies={"DRIFT_POLICY_FRAUD_MODEL": drift_policy}
)

## 5. Creating a Workflow

Workflows are the heart of the Control Plane's orchestration logic. A workflow defines a sequence of steps involving agents and policies.

This `drift_management_workflow` does the following:
1.  Receives an `observed_drift` value.
2.  Uses the `PolicyEngine` to check if the drift policy is triggered.
3.  If triggered, it calls the `AtlasAgent` to start a retraining pipeline and the `IntelligenceAgent` to generate a summary.

In [ ]:
engine = WorkflowEngine(
    logger=logger,
    policy_engine=policy_engine,
)

def drift_management_workflow(context, policy_engine, logger, run_id):
    observed_drift = context.get("observed_drift", 0.0)

    policy_result = policy_engine.evaluate(
        "DRIFT_POLICY_FRAUD_MODEL",
        context={"observed_drift": observed_drift},
    )

    steps = {
        "observed_drift": observed_drift,
        "policy_result": policy_result,
        "actions_taken": [],
    }

    if policy_result["should_act"]:
        if "trigger_retrain" in policy_result["actions"]:
            atlas_result = atlas_agent.run(action="trigger_retrain")
            steps["actions_taken"].append(
                {"type": "atlas_retrain", "result": atlas_result}
            )

        intel_result = intel_agent.run(
            query=(
                f"Observed model drift of {observed_drift}. "
                f"Policy triggered retraining. "
                f"Explain this decision to an executive."
            )
        )
        steps["actions_taken"].append(
            {"type": "intel_summary", "result": intel_result}
        )

    return steps

engine.register_workflow(
    name="fraud_drift_management",
    description="End-to-end fraud model drift management workflow.",
    handler=drift_management_workflow,
)

## 6. Run the Workflow

Now, let's run the workflow. We'll provide a `demo_context` with an `observed_drift` of `0.22`, which is higher than our policy's threshold of `0.15`.

In [ ]:
demo_context = {"observed_drift": 0.22}

result = engine.run_workflow(
    name="fraud_drift_management",
    context=demo_context,
)

## 7. View the Results

The final output shows the complete, auditable result of the workflow run. We can see the observed drift, the policy decision, and the actions taken by the orchestrated agents.

In [ ]:
import json
print("=== CONTROL PLANE WORKFLOW RESULT ===")
print(json.dumps(result, indent=2))

## Next Steps

This demo provides a simple but powerful example of an AI Control Plane.

From here, you can:
- **Replace the stubs:** Integrate your real MLOps and Intelligence Agent implementations.
- **Expand the policies:** Add more sophisticated rules for security, compliance, or operations.
- **Create new workflows:** Build new orchestration logic for different use cases like supply chain anomaly detection or financial compliance.